# Deep-Learning Agents
## Planning Stocks

In [15]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [43]:
import os
import autogen
from autogen import AssistantAgent, UserProxyAgent
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

llm_config = {"model": "gpt-3.5-turbo"}
task = "Write a blog post about the implementation and benefits of using GitOps with ArgoCD for continuous delivery in Kubernetes environments. "\
"Include specific examples and case studies to illustrate the improvements in deployment speed and reliability. Today's date is 2024-04-23."

### Group chat

- User_proxy or Admin
- Planner
- Engineer
- Executor
- Writer

In [44]:
! pip install -q -U pyautogen autogen
import autogen

In [45]:
## ADMIN Agent
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Assign the task and send instructions to the engineer to refine the DevOps automation script.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS"
)

## PLANNER Agent
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a DevOps task, please determine what information is needed to complete the task. "
    "After each step is done by others, check the progress and instruct the remaining steps. If a step fails, try to workaround.",
    description="Planner. Given a DevOps task, determine what information is needed to complete the task. "
    "After each step is done by others, check the progress and instruct the remaining steps.",
    llm_config=llm_config,
)

## ENGINEER Agent
engineer = autogen.ConversableAgent(
    name="Engineer",
    llm_config=llm_config,
    description="A DevOps engineer that writes automation scripts and configurations"
    " based on the plan provided by the planner.",
)

## EXECUTOR Agent
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the scripts and configurations"
    " written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        # "user_docker": False,
    },
)

## WRITER Agent
writer = autogen.ConversableAgent(
    name="Writer",
    system_message="Writer. Please document the DevOps processes and automation scripts in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your documentation.",
    description="Writer."
    "Writer. Document the DevOps processes and automation scripts"
    " based on the execution results and take feedback from the admin to refine the documentation.",
)

### Define the group chat

In [46]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, planner, engineer, executor, writer],
    messages=[],
    max_round=10,
)

In [47]:
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
)

### Start the group chat

In [48]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blog post about the implementation and benefits of using GitOps with ArgoCD for continuous delivery in Kubernetes environments. Include specific examples and case studies to illustrate the improvements in deployment speed and reliability. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Writer (to chat_manager):



--------------------------------------------------------------------------------
Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Admin (to chat_manager):

Hello! 

I would like you to write a blog post about the implementation and benefits of using GitOps with ArgoCD for continuous delivery in Kubernetes environments. Please include specific examples and case studies to illustrate the improvements in deployment speed and r

### Add a Speaker

In [49]:
## ADMIN Agent
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Assign the task and send instructions to the engineer to refine the DevOps automation script.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS"
)

## PLANNER Agent
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a DevOps task, please determine what information is needed to complete the task. "
    "After each step is done by others, check the progress and instruct the remaining steps. If a step fails, try to workaround.",
    description="Planner. Given a DevOps task, determine what information is needed to complete the task. "
    "After each step is done by others, check the progress and instruct the remaining steps.",
    llm_config=llm_config,
)

## ENGINEER Agent
engineer = autogen.ConversableAgent(
    name="Engineer",
    llm_config=llm_config,
    description="A DevOps engineer that writes automation scripts and configurations"
    " based on the plan provided by the planner.",
)

## EXECUTOR Agent
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the scripts and configurations"
    " written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        # "user_docker": False,
    },
)

## WRITER Agent
writer = autogen.ConversableAgent(
    name="Writer",
    system_message="Writer. Please document the DevOps processes and automation scripts in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your documentation.",
    description="Writer."
    "Writer. Document the DevOps processes and automation scripts"
    " based on the execution results and take feedback from the admin to refine the documentation.",
)

In [50]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [51]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blog post about the implementation and benefits of using GitOps with ArgoCD for continuous delivery in Kubernetes environments. Include specific examples and case studies to illustrate the improvements in deployment speed and reliability. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Writer (to chat_manager):



--------------------------------------------------------------------------------
Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Admin (to chat_manager):

Hello! 

I would like you to write a blog post about the implementation and benefits of using GitOps with ArgoCD for continuous delivery in Kubernetes environments. Please include specific examples and case studies to illustrate the improvements in deployment speed and r